In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = './dataset/training'
test_dir = './dataset/testing'

train_datagen = ImageDataGenerator(
    rescale=1./255,  
    rotation_range=15,  
    width_shift_range=0.1,  
    height_shift_range=0.1,  
    shear_range=0.1,  
    zoom_range=0.1,  
    horizontal_flip=True,  
    fill_mode='nearest'  
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  
    batch_size=32,
    class_mode='categorical',  
    color_mode='grayscale' 
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [16]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# class weights to handle imbalanced data
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}


#### Adjusting CapsNet Architecture for Detection
##### Capsule Network Design

In [20]:
import numpy as np
from tensorflow.keras import layers, models, Input
from capsule_layers import CapsuleLayer, Mask, margin_loss, squash

def CapsNet(input_shape, n_classes, routings):
    x = layers.Input(shape=input_shape)

    # Conv layer
    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu')(x)
    conv1 = layers.BatchNormalization()(conv1)
    
    # PrimaryCaps layer
    primarycaps = CapsuleLayer(num_capsule=32, dim_capsule=8, routings=1)(conv1)

    # DigitCaps layer
    digitcaps = CapsuleLayer(num_capsule=n_classes, dim_capsule=16, routings=routings, name='digitcaps')(primarycaps)

    # Decoder network
    y = layers.Input(shape=(n_classes,))
    masked = Mask()([digitcaps, y])
    
    # Flatten the masked output before feeding into the decoder
    flattened = layers.Flatten()(masked)

    decoder = models.Sequential([
        layers.Dense(512, activation='relu', input_dim=n_classes * 16),  # Adjusted input dimension
        layers.Dense(1024, activation='relu'),
        layers.Dense(np.prod(input_shape), activation='sigmoid'),
        layers.Reshape(target_shape=input_shape)
    ], name='decoder')

    decoded = decoder(flattened)

    model = models.Model([x, y], [digitcaps, decoded])
    model.compile(optimizer='adam',
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., 0.392],
                  metrics={'capsnet': 'accuracy'})

    return model


In [21]:
input_shape = (128, 128, 1)  
n_classes = 4  
routings = 3  

model = CapsNet(input_shape, n_classes, routings)
print(model.summary())

epochs = 50 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
]



Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 120, 120,  │     20,992 │ input_layer_9[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │      1,024 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ capsule_layer_3     │ (None, 32, 8)     │     65,536 │ batch_normalizat… │
│ (CapsuleLayer)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ digitcaps           │ (None, 4, 16)     │        512 │ capsule_layer_3[… │
│ (CapsuleLayer)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_10      │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mask_3 (Mask)       │ (None, 4, 16)     │          0 │ digitcaps[0][0],  │
│                     │                   │            │ input_layer_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 64)        │          0 │ mask_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder             │ (None, 128, 128,  │ 17,352,192 │ flatten[0][0]     │
│ (Sequential)        │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,440,256 (66.53 MB)

 Trainable params: 17,439,744 (66.53 MB)

 Non-trainable params: 512 (2.00 KB)

None


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=best_model.h5